In [1]:
import io
import numpy as np
import tensorflow as tf
from librosa import effects

from utils import spectrogram2wav, plot_test_alignment

from train import Graph
from hyperparams import Hyperparams as hp
import tqdm
from IPython.display import Audio

In [2]:
def load_vocab():
    char2idx = {char: idx for idx, char in enumerate(hp.vocab)}
    idx2char = {idx: char for idx, char in enumerate(hp.vocab)}
    return char2idx, idx2char

In [3]:
def get_EOS_index(text):
    # Load vocab
    char2idx, idx2char = load_vocab()

    _text = np.array([idx2char[t] for t in text])
    return np.argmax(_text == hp.EOS_char)

In [4]:
char2idx, idx2char = load_vocab()

In [5]:
def get_EOS_index(text):
    # text : texto codificado en numeros
    # Load vocab
    char2idx, idx2char = load_vocab()

    _text = np.array([idx2char[t] for t in text])
    return np.argmax(_text == hp.EOS_char)

def get_EOS_fire(alignment,text):
    EOS_index = get_EOS_index(text)
    text_max_indicies = np.argmax(alignment,axis = 0)
    r = []
    for i,max_index in enumerate(text_max_indicies):
        if max_index == EOS_index:
            r.append(i)
    if not len(r) == 0:
        return max(r)
    return None

import scipy
def save_wav(wav, path, sr):
  wav *= 32767 / max(0.01, np.max(np.abs(wav)))
  scipy.io.wavfile.write(path, sr, wav.astype(np.int16))

In [6]:
g = Graph(mode="synthesize"); print("Graph loaded")

saver = tf.train.Saver()

session = tf.Session()
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(session, tf.train.latest_checkpoint(hp.syn_logdir)); print("Restored!")

Graph loaded
INFO:tensorflow:Restoring parameters from logdir_es\model_gs_400k
Restored!


In [122]:
text = 'Solo se logran estos resultados con un equipo profesional que respalda y asegura el éxito de cada reto al que nos enfrentamos y  Confío en ustedes y les reitero mis felicitaciones y el agradecimiento por su especial esfuerzo. '.strip()+u"␃"
text

'Solo se logran estos resultados con un equipo profesional que respalda y asegura el éxito de cada reto al que nos enfrentamos y  Confío en ustedes y les reitero mis felicitaciones y el agradecimiento por su especial esfuerzo.␃'

In [123]:
len(text_encode)

224

In [124]:
text_encode = [char2idx[char] for char in text]
print(text_encode)

[29, 51, 48, 51, 2, 55, 41, 2, 48, 51, 43, 54, 37, 50, 2, 41, 55, 56, 51, 55, 2, 54, 41, 55, 57, 48, 56, 37, 40, 51, 55, 2, 39, 51, 50, 2, 57, 50, 2, 41, 53, 57, 45, 52, 51, 2, 52, 54, 51, 42, 41, 55, 45, 51, 50, 37, 48, 2, 53, 57, 41, 2, 54, 41, 55, 52, 37, 48, 40, 37, 2, 61, 2, 37, 55, 41, 43, 57, 54, 37, 2, 41, 48, 2, 74, 60, 45, 56, 51, 2, 40, 41, 2, 39, 37, 40, 37, 2, 54, 41, 56, 51, 2, 37, 48, 2, 53, 57, 41, 2, 50, 51, 55, 2, 41, 50, 42, 54, 41, 50, 56, 37, 49, 51, 55, 2, 61, 2, 2, 13, 51, 50, 42, 76, 51, 2, 41, 50, 2, 57, 55, 56, 41, 40, 41, 55, 2, 61, 2, 48, 41, 55, 2, 54, 41, 45, 56, 41, 54, 51, 2, 49, 45, 55, 2, 42, 41, 48, 45, 39, 45, 56, 37, 39, 45, 51, 50, 41, 55, 2, 61, 2, 41, 48, 2, 37, 43, 54, 37, 40, 41, 39, 45, 49, 45, 41, 50, 56, 51, 2, 52, 51, 54, 2, 55, 57, 2, 41, 55, 52, 41, 39, 45, 37, 48, 2, 41, 55, 42, 57, 41, 54, 62, 51, 7, 1]


In [125]:
y_hat = np.zeros((1, 250, hp.n_mels*hp.r), np.float32)

In [126]:
for j in tqdm.tqdm(range(250)):
    _y_hat = session.run(g.y_hat, {g.x: [text_encode], g.y: y_hat})
    y_hat[:, j, :] = _y_hat[:, j, :]

100%|████████████████████████████████████████| 250/250 [00:32<00:00,  7.72it/s]


In [127]:
## mag
mag = session.run(g.z_hat, {g.y_hat: y_hat})

In [128]:
audio = spectrogram2wav(mag[0,:,:],1)

In [129]:
out = io.BytesIO()
save_wav(audio, out, hp.sr)
Audio(out.getvalue(),autoplay=True)

## Analizando Endpoint

In [30]:
import librosa
from scipy import signal
import copy

In [31]:
def griffin_lim(spectrogram):
    '''Applies Griffin-Lim's raw.
    '''
    X_best = copy.deepcopy(spectrogram)
    for i in range(hp.n_iter):
        X_t = invert_spectrogram(X_best)
        est = librosa.stft(X_t, hp.n_fft, hp.hop_length, win_length=hp.win_length)
        phase = est / np.maximum(1e-8, np.abs(est))
        X_best = spectrogram * phase
    X_t = invert_spectrogram(X_best)
    y = np.real(X_t)

    return y

In [32]:
def invert_spectrogram(spectrogram):
    '''
    spectrogram: [f, t]
    '''
    return librosa.istft(spectrogram, hp.hop_length, win_length=hp.win_length, window="hann")

In [117]:
mag = mag[0,:,:].T
# de-noramlize
mag = (np.clip(mag, 0, 1) * hp.max_db) - hp.max_db + hp.ref_db
# to amplitude
mag = np.power(10.0, mag * 0.05)
# wav reconstruction
wav = griffin_lim(mag)
# de-preemphasis
wav = signal.lfilter([1], [1, -hp.preemphasis], wav)

In [118]:
#trim
frame_length = int(hp.sr * 0.8)
hop_length = int(frame_length / 4)

In [119]:
endpoints = librosa.effects.split(wav, frame_length=frame_length,
                               hop_length=hop_length,
                               top_db=40)[:, 1]

In [120]:
print(endpoints)
ep = endpoints[0]
print(ep)
print(wav.shape)

[149940]
149940
(343475,)


In [121]:
out = io.BytesIO()
save_wav(wav, out, hp.sr)
Audio(out.getvalue(),autoplay=True)